In [7]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
from scipy.integrate import quad, quad_vec
import matplotlib.pyplot as plt
from scipy import linalg
from tqdm.notebook import tqdm
import qutip as qt
import sys
from numba import jit
from qutip.nonmarkov.heom import (
    HEOMSolver,
    DrudeLorentzPadeBath,
    BathExponent,
    DrudeLorentzBath,
)

lam = 0.05
ϵ = 1e-3


@jit(nopython=True)
def f(alpha, t1=1):
    return alpha / (t1 * np.sqrt(2 * np.pi))


def bose(ν, T):
    if T == 0:
        return 0
    return 1 / (np.exp(ν / T) - 1)


@jit(nopython=True)
def spectral_density(w, lam, gamma):
    return lam * w**3 * np.exp(-abs(w / gamma))


γmama = (
    lambda lam, alpha, w, T, t, wc: np.exp(1j * (2 * alpha) + 4 * f(alpha) * t)
    * (spectral_density(w, lam, wc) / (w**2 - 4 * f(alpha)))
    * (2 * bose(w, T) + 1)
)
γmeme = (
    lambda lam, alpha, w, T, t, wc: np.exp(-1j * (2 * alpha) + 4 * f(alpha) * t)
    * (spectral_density(w, lam, wc) / (w**2 - 4 * f(alpha)))
    * (2 * bose(w, T) + 1)
)
γmame = (
    lambda lam, alpha, w, T, t, wc: spectral_density(w, lam, wc)
    * (
        bose(w, T) / (2 * f(alpha) - w) ** 2
        + (bose(w, T) + 1) / (2 * f(alpha) + w) ** 2
    )
    if ((2 * f(alpha) - w) > 0) and ((2 * f(alpha) + w) > 0)
    else 0
)
γmema = (
    lambda lam, alpha, w, T, t, wc: spectral_density(w, lam, wc)
    * (
        bose(w, T) / (2 * f(alpha) + w) ** 2
        + (bose(w, T) + 1) / (2 * f(alpha) - w) ** 2
    )
    if ((2 * f(alpha) - w) > 0) and ((2 * f(alpha) + w) > 0)
    else 0
)

Γplus = lambda T, t, wc, alpha: quad_vec(
    lambda w: γmame(lam, alpha, w, T, t, wc),
    0,
    np.Inf,
    epsabs=ϵ,
    epsrel=ϵ,
    quadrature="gk21",
)[0]
Γminus = lambda T, t, wc, alpha: quad_vec(
    lambda w: γmema(lam, alpha, w, T, t, wc),
    0,
    np.Inf,
    epsabs=ϵ,
    epsrel=ϵ,
    quadrature="gk21",
)[0]
Γplusminus = lambda T, t, wc, alpha: quad_vec(
    lambda w: γmama(lam, alpha, w, T, t, wc),
    0,
    np.Inf,
    epsabs=ϵ,
    epsrel=ϵ,
    quadrature="gk21",
)[0]
Γminusplus = lambda T, t, wc, alpha: quad_vec(
    lambda w: γmeme(lam, alpha, w, T, t, wc),
    0,
    np.Inf,
    epsabs=ϵ,
    epsrel=ϵ,
    quadrature="gk21",
)[0]


def M(T, t, wc, alpha=1):
    gplus = Γplus(T, t, wc, alpha)
    gminus = np.conj(Γminus(T, t, wc, alpha))
    gamma = gplus + gminus
    plusminus = Γplusminus(T, t, wc, alpha)

    matriz = np.array(
        [
            [-gamma, 0, 0],
            [
                0,
                -np.real(plusminus) - (gamma / 2),
                -np.imag(plusminus),
            ],
            [
                0,
                -np.imag(plusminus),
                np.real(plusminus) - (gamma / 2),
            ],
        ]
    )
    r = [
        (gplus - gminus) / 2,
        0,
        0,
    ]
    return matriz, r


def dynamics(T, t, wc, a, alpha=1):
    s1 = np.array([[0, 1], [1, 0]])
    s2 = np.array([[0, -1j], [1j, 0]])
    s3 = np.array([[1, 0], [0, -1]])
    if t == 0:
        return (np.eye(2) / 2) + a[0] * s3 + a[2] * s2 + a[1] * s1
    m, r = M(T, t, wc, alpha=alpha)
    exponential = linalg.expm(m)
    pauli_coeff = (exponential - np.eye(3)) @ np.linalg.inv(m) @ r
    with_pauli = pauli_coeff[0] * s3 + pauli_coeff[1] * s1 + pauli_coeff[2] * s2
    transient = exponential @ a
    transient_base = transient[0] * s3 + transient[1] * s1 + transient[2] * s2
    total = (np.eye(2) / 2) + transient_base + with_pauli
    return total


def cumulant(t, T, a, b, c, wc, alpha, lam=lam, ϵ=ϵ, dm=True, l=10, ls=False):
    t = np.linspace(0.01, t, l)
    data = [
        dynamics(T, i, wc, [a, b, c], alpha)
        for i in tqdm(t, desc=f"Computing for all t, currently on ")
    ]
    if dm:
        return data
    ρ11 = np.array([data[i][0, 0] for i in range(len(data))])
    ρ12 = np.array([data[i][0, 1] for i in range(len(data))])
    ρ21 = np.array([data[i][1, 0] for i in range(len(data))])
    ρ22 = np.array([data[i][1, 1] for i in range(len(data))])
    return ρ11, ρ12, ρ21, ρ22


ModuleNotFoundError: No module named 'numba'